In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL
import json

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


(841.68, 1190.8799999999999)
0.13.2


In [2]:
jsonfilepath = '2025\DDS2\\assessment-data.json'
folder, file, ext = getFolderandFileName(jsonfilepath)
with open(jsonfilepath) as f:
    data = json.load(f)
# display(data)
print(json.dumps(data, indent=4))

{
    "success": true,
    "result": [
        {
            "assessment_id": 383,
            "student_name": "Jonathan Gershon",
            "student_number": 1391589,
            "assessor_name": "Melinda Johansson",
            "date": "2025-12-17",
            "cohort": "DDS2",
            "subject": "DENT90146",
            "type": "Simulation",
            "student_data": {
                "student_524_MC1": "Yes",
                "student_524_MC2": "Yes",
                "student_524_MC3": "Not Assessed",
                "student_524_MC4": "Not Assessed",
                "student_524_MC5": "Not Assessed",
                "student_524_MC6": "Not Assessed",
                "student_524_MC7": "Yes",
                "student_524_MC8": "No",
                "student_524_MC9": "No",
                "student_578_MC1": "Yes",
                "student_578_MC2": "No",
                "student_578_MC3": "No",
                "student_578_MC4": "No",
                "student_578_MC5": "No"

In [3]:
# Convert to a structured DataFrame per date
data_dict = {}
df_dict = {}

for entry in data["result"]:
    date = entry["date"]
    flattened_entry = {
        "student_name": entry["student_name"],
        "assessor_name": entry["assessor_name"],
        "date": entry["date"],
        "cohort": entry["cohort"],
        "subject": entry["subject"],
        "type": entry["type"],
        **entry["student_data"],
        **entry["supervisor_data"]
    }
    
    if date not in data_dict:
        data_dict[date] = []
    
    data_dict[date].append(flattened_entry)

# Create and display separate DataFrames per date
for date, records in data_dict.items():
    df = pd.DataFrame(records)
    df_dict[date] = df
#     df.to_excel(f"{folder}\{date}.xlsx", index=False)
# # save all in a single file with different sheets
# with pd.ExcelWriter(f"{folder}\\all_data.xlsx") as writer:
#     for date, df in df_dict.items():
#         df.to_excel(writer, sheet_name=date, index=False)

# # create one with all data
# df_all = pd.concat(df_dict.values(), ignore_index=True)
# with pd.ExcelWriter(f"{folder}\\all_data.xlsx", engine= 'openpyxl', mode= 'a') as writer:
#     df_all.to_excel(writer, index=False, sheet_name='all_data')

## DDS2

In [60]:
workbookpath = '2025\DDS2\\Michael\\all_data.xlsx'
workbook, folder, file = loadWorkbook(workbookpath)
allData = pd.read_excel(workbookpath, sheet_name='all_data')
mcCols = [col for col in allData.columns if 'MC' in col]
display(allData.head())
# # get value counts for each column
# for col in mcCols:
#     print(allData[col].value_counts())

'Loaded workbook: 2025\\DDS2\\Michael | all_data | .xlsx'
("Workbook sheets: ['2025-03-03', '2025-02-27', '2025-02-25', '2025-02-24', "
 "'2025-02-20', '2025-02-17', '2025-02-13', '2025-02-10', '2025-02-06', "
 "'2025-02-03', '2025-01-30', '2025-01-29', 'all_data']")


,student_name,assessor_name,date,cohort,subject,type,student_524_MC1,student_524_MC2,student_524_MC3,student_524_MC4,student_524_MC5,student_524_MC6,student_524_MC7,student_524_MC8,student_524_MC9,student_578_MC1,student_578_MC2,student_578_MC3,student_578_MC4,student_578_MC5,student_524_MC10,student_524_MC11,student_524_MC12,student_524_MC13,student_524_MC14,student_524_MC15,student_524_MC16,student_524_MC17,student_feedback,time_mgmt,entrustment,communication,professionalism,assessor_524_MC1,assessor_524_MC2,assessor_524_MC3,assessor_524_MC4,assessor_524_MC5,assessor_524_MC6,assessor_524_MC7,assessor_524_MC8,assessor_524_MC9,assessor_578_MC1,assessor_578_MC2,assessor_578_MC3,assessor_578_MC4,assessor_578_MC5,assessor_524_MC10,assessor_524_MC11,assessor_524_MC12,assessor_524_MC13,assessor_524_MC14,assessor_524_MC15,assessor_524_MC16,assessor_524_MC17,assessor_feedback,clinical_incident,assessor_positioning_MC1,assessor_positioning_MC2,student_533_MC1,assessor_533_MC1,student_011_MC1,student_011_MC2,student_011_MC3,student_011_MC4,student_011_MC5,student_011_MC6,student_011_MC7,student_011_MC8,student_011_MC9,student_011_MC10,student_011_MC11,student_011_MC12,student_011_MC13,student_011_MC14,student_011_MC15,student_011_MC16,student_011_MC17,student_011_MC18,assessor_011_MC1,assessor_011_MC2,assessor_011_MC3,assessor_011_MC4,assessor_011_MC5,assessor_011_MC6,assessor_011_MC7,assessor_011_MC8,assessor_011_MC9,assessor_011_MC10,assessor_011_MC11,assessor_011_MC12,assessor_011_MC13,assessor_011_MC14,assessor_011_MC15,assessor_011_MC16,assessor_011_MC17,assessor_011_MC18,patient_complexity,assessor_Consent_MC1,assessor_Record_keeping_MC1,assessor_infection_control_MC1,assessor_infection_control_MC2,student_022_MC1,student_022_MC2,student_022_MC3,student_022_MC4,student_022_MC5,student_022_MC6,student_022_MC7,student_022_MC8,student_221_MC1,student_221_MC2,student_221_MC3,student_221_MC4,student_221_MC5,student_221_MC6,student_221_MC7,student_221_MC8,assessor_022_MC1,assessor_022_MC2,assessor_022_MC3,assessor_022_MC4,assessor_022_MC5,assessor_022_MC6,assessor_022_MC7,assessor_022_MC8,assessor_221_MC1,assessor_221_MC2,assessor_221_MC3,assessor_221_MC4,assessor_221_MC5,assessor_221_MC6,assessor_221_MC7,assessor_221_MC8,student_012_MC1,student_012_MC2,student_012_MC3,student_012_MC4,student_012_MC5,student_012_MC6,student_012_MC7,student_012_MC8,student_012_MC9,student_012_MC10,student_012_MC11,student_012_MC12,student_012_MC13,student_012_MC14,student_012_MC15,student_012_MC16,student_012_MC17,student_012_MC18,assessor_012_MC1,assessor_012_MC2,assessor_012_MC3,assessor_012_MC4,assessor_012_MC5,assessor_012_MC6,assessor_012_MC7,assessor_012_MC8,assessor_012_MC9,assessor_012_MC10,assessor_012_MC11,assessor_012_MC12,assessor_012_MC13,assessor_012_MC14,assessor_012_MC15,assessor_012_MC16,assessor_012_MC17,assessor_012_MC18,student_013_MC1,student_013_MC2,student_013_MC3,student_013_MC4,student_013_MC5,student_013_MC6,student_013_MC7,student_013_MC8,assessor_013_MC1,assessor_013_MC2,assessor_013_MC3,assessor_013_MC4,assessor_013_MC5,assessor_013_MC6,assessor_013_MC7,assessor_013_MC8,student_014_MC1,student_014_MC2,student_014_MC3,student_014_MC4,assessor_014_MC1,assessor_014_MC2,assessor_014_MC3,assessor_014_MC4,student_577_MC1,student_577_MC2,student_577_MC3,student_577_MC4,assessor_577_MC1,assessor_577_MC2,assessor_577_MC3,assessor_577_MC4,student_LA-2025_MC1,student_LA-2025_MC2,student_LA-2025_MC3,student_LA-2025_MC4,student_LA-2025_MC5,student_LA-2025_MC6,student_LA-2025_MC7,student_LA-2025_MC8,student_LA-2025_MC9,student_LA-2025_MC10,student_LA-2025_MC11,student_LA-2025_MC12,student_LA-2025_MC13,assessor_LA-2025_MC1,assessor_LA-2025_MC2,assessor_LA-2025_MC3,assessor_LA-2025_MC4,assessor_LA-2025_MC5,assessor_LA-2025_MC6,assessor_LA-2025_MC7,assessor_LA-2025_MC8,assessor_LA-2025_MC9,assessor_LA-2025_MC10,assessor_LA-2025_MC11,assessor_LA-2025_MC12,assessor_LA-2025_MC13,student_165_MC1,student_165_MC2,student_165_MC3,student_165_MC4,st

In [61]:
michaelDates = ['2025-02-03', '2025-02-10','2025-02-17', '2025-02-24']
michaelData = allData[allData['date'].isin(michaelDates)]
# drop empty columns
michaelData = michaelData.dropna(axis=1, how='all')
# display(michaelData.head())
df = michaelData.copy()
# df = df.loc[:, ~df.columns.str.contains("533")]
# df = df.loc[:, ~df.columns.str.contains("578")]
# display(michaelData.head(10))
#  Define the new column order
# column_order = [
#     # Basic Info
#     "student_name", "assessor_name", "date", "cohort", "subject", "type",
    
#     # Student MC Scores
#     "student_524_MC1", "student_524_MC2", "student_524_MC3", "student_524_MC4", "student_524_MC5",
#     "student_524_MC6", "student_524_MC7", "student_524_MC8", "student_524_MC9", "student_524_MC10",
#     "student_524_MC11", "student_524_MC12", "student_524_MC13", "student_524_MC14", "student_524_MC15",
#     "student_524_MC16", "student_524_MC17", 
#     "student_578_MC1", "student_578_MC2", "student_578_MC3", "student_578_MC4", "student_578_MC5",
#     "student_533_MC1", "student_533_MC2", "student_533_MC3", "student_533_MC4", "student_533_MC5",
#     "student_533_MC6", "student_533_MC7", "student_533_MC8", "student_533_MC9", "student_533_MC10",
#     "student_533_MC11", "student_533_MC12", "student_533_MC13", "student_533_MC14", "student_533_MC15",
#     "student_533_MC16", "student_533_MC17",
#     "student_577_MC1", "student_577_MC2", "student_577_MC3", "student_577_MC4",

#     # Assessor MC Scores
#     "assessor_524_MC1", "assessor_524_MC2", "assessor_524_MC3", "assessor_524_MC4", "assessor_524_MC5",
#     "assessor_524_MC6", "assessor_524_MC7", "assessor_524_MC8", "assessor_524_MC9", "assessor_524_MC10",
#     "assessor_524_MC11", "assessor_524_MC12", "assessor_524_MC13", "assessor_524_MC14", "assessor_524_MC15",
#     "assessor_524_MC16", "assessor_524_MC17", 
#     "assessor_578_MC1", "assessor_578_MC2", "assessor_578_MC3", "assessor_578_MC4", "assessor_578_MC5",
#     "assessor_533_MC1", "assessor_533_MC2", "assessor_533_MC3", "assessor_533_MC4", "assessor_533_MC5",
#     "assessor_533_MC6", "assessor_533_MC7", "assessor_533_MC8", "assessor_533_MC9", "assessor_533_MC10",
#     "assessor_533_MC11", "assessor_533_MC12", "assessor_533_MC13", "assessor_533_MC14", "assessor_533_MC15",
#     "assessor_533_MC16", "assessor_533_MC17",
#     "assessor_577_MC1", "assessor_577_MC2", "assessor_577_MC3", "assessor_577_MC4",

#     # Assessor Feedback & Other Info
#     "assessor_feedback", "clinical_incident", "assessor_positioning_MC1", "assessor_positioning_MC2",
    
#     # Student Feedback
#     "student_feedback",

#     # Move These to the End
#     "time_mgmt", "entrustment", "communication", "professionalism"
# ]

# # Reorder the columns
# df = df[column_order]



In [62]:
df.fillna('', inplace=True)
with open('2025\sim_checklists.json') as f:
    simChecklist = json.load(f)
    simChecklist = {item['key']: item for item in simChecklist['result']}     # can get full name by key fields MCx
with open('2025\clinic_checklists.json') as f:
    clinicChecklist = json.load(f)
    clinicChecklist = {item['key']: item for item in clinicChecklist['result']}

pprint(simChecklist, indent=4)
pprint(clinicChecklist, indent=4)
pecItems  = ['positioning', 'Consent', 'Record', 'infection']
rubricCols = ['time_mgmt', 'communication', 'professionalism', 'entrustment']
mcCols = [col for col in df.columns if 'MC' in col and 'positioning' not in col and 'Consent' not in col and 'Record' not in col and 'infection' not in col]
pecCols = [col for col in df.columns if 'positioning' in col or 'Consent' in col or 'Record' in col or 'infection' in col]
studentmcCols = [col for col in mcCols if 'student' in col]
assessormcCols = [col for col in mcCols if 'assessor' in col]

otherCols = [col for col in df.columns if col not in mcCols and col not in rubricCols and col not in pecCols]
print(otherCols)
otherfullnamedict = {'student_name': 'Student Name', 'assessor_name': 'Assessor Name', 'date': 'Date', 'cohort': 'Cohort', 'subject': 'Subject', 'type': 'Type',
                     'time_mgmt': 'Time Management', 'communication': 'Communication', 'professionalism': 'Professionalism', 'entrustment': 'Entrustment',
                     'student_feedback': 'Student Feedback', 'assessor_feedback': 'Assessor Feedback', 'clinical_incident': 'Clinical Incident',
                     'patient_complexity': 'Complexity', 'Consent': 'Consent'}
mcColDict = {}
for col in mcCols:
    key = col.split('_')[1]
    if key not in mcColDict:
        mcColDict[key] = []
    mcColDict[key].append(col)
def get_items(row):
    items = []
    for key, cols in mcColDict.items():
        if any((not pd.isna(row[col])) and (row[col] not in ['', 'nan']) for col in cols):
            if key in pecItems:
                continue
            items.append(key)
    return items

df['Items'] = df.apply(get_items, axis=1)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
# remove rows where all mc columns are empty
df = df[df['Items'].apply(lambda x: len(x) > 0)]
display(df.head())
 

{   '114-hs': {   'fields': {   'MC1': 'Selects the curette for the specific '
                                       'area',
                                'MC10': 'Minimises damage to soft tissues',
                                'MC2': 'Checks instrument sharpness',
                                'MC3': 'Maintains a modified pen grasp during '
                                       'debridement',
                                'MC4': 'Adapts the instrument to the tooth '
                                       'surface',
                                'MC5': 'Debrides with continuous, overlapping '
                                       'and controlled strokes away from the '
                                       'base of the pocket.',
                                'MC6': 'Uses stable finger rest in the same '
                                       'quadrant throughout the procedure',
                                'MC7': 'Applies lateral pressure during '
                  

,student_name,assessor_name,date,cohort,subject,type,student_524_MC1,student_524_MC2,student_524_MC3,student_524_MC4,student_524_MC5,student_524_MC6,student_524_MC7,student_524_MC8,student_524_MC9,student_578_MC1,student_578_MC2,student_578_MC3,student_578_MC4,student_578_MC5,student_524_MC10,student_524_MC11,student_524_MC12,student_524_MC13,student_524_MC14,student_524_MC15,student_524_MC16,student_524_MC17,student_feedback,time_mgmt,entrustment,communication,professionalism,assessor_524_MC1,assessor_524_MC2,assessor_524_MC3,assessor_524_MC4,assessor_524_MC5,assessor_524_MC6,assessor_524_MC7,assessor_524_MC8,assessor_524_MC9,assessor_578_MC1,assessor_578_MC2,assessor_578_MC3,assessor_578_MC4,assessor_578_MC5,assessor_524_MC10,assessor_524_MC11,assessor_524_MC12,assessor_524_MC13,assessor_524_MC14,assessor_524_MC15,assessor_524_MC16,assessor_524_MC17,assessor_feedback,clinical_incident,assessor_positioning_MC1,assessor_positioning_MC2,student_533_MC1,assessor_533_MC1,student_577_MC1,student_577_MC2,student_577_MC3,student_577_MC4,assessor_577_MC1,assessor_577_MC2,assessor_577_MC3,assessor_577_MC4,student_533_MC2,student_533_MC3,student_533_MC4,student_533_MC5,student_533_MC6,student_533_MC7,student_533_MC8,student_533_MC9,student_533_MC10,student_533_MC11,student_533_MC12,student_533_MC13,student_533_MC14,student_533_MC15,student_533_MC16,student_533_MC17,assessor_533_MC2,assessor_533_MC3,assessor_533_MC4,assessor_533_MC5,assessor_533_MC6,assessor_533_MC7,assessor_533_MC8,assessor_533_MC9,assessor_533_MC10,assessor_533_MC11,assessor_533_MC12,assessor_533_MC13,assessor_533_MC14,assessor_533_MC15,assessor_533_MC16,assessor_533_MC17,Items
641,Bella Dong,Melinda Johansson,2025-02-03,DDS2,DENT90146,Simulation,,,,,,,,,,,,,,,,,,,,,,,,1.0,1.0,2.0,2.0,,,,,,,,,,,,,,,,,,,,,,,Suggest placing frame on top of dam to prevent...,,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,No,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,Yes,No,Yes,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,"[533, 577]"
573,Mehrana Rahimpour,Michael Wylie,2025-02-03,DDS2,DENT90146,Simulation,,,,,,,,,,,,,,,,,,,,,,,I was comfortable with the MO prep as it was a...,2.0,2.0,2.0,2.0,,,,,,,,,,,,,,,,,,,,,,,Over widening buccal to palatal. Damage 15D fr...,,Yes,Yes,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,No,No,Not Reviewed,Not Reviewed,Yes,Yes,Yes,No,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,Yes,Yes,"[533, 577]"
574,Ellen Sampsonidis,Michelle Luu,2025-02-03,DDS2,DENT90146,Simulation,,,,,,,,,,,,,,,,,,,,,,,Something I think I did well was time manageme...,1.0,2.0,2.0,2.0,,,,,,,,,,,,,,,,,,,,,,,"Good isolation, clamp selection, mindful of ne...",,Yes,No,Yes,Yes,No,No,Not Assessed,Not Assessed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,No,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,Yes,Yes,Yes,Yes,No,Yes,No,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,Yes,Yes,Yes,"[533, 577]"
575,Jennifer Lee,Carl Lakkis,2025-02-03,DDS2,DENT90146,Simulation,,,,,,,,,,,,,,,,,,,,,,,533_x000D_\n- Isolation: small tear between th...,1.0,1.0,2.0,2.0,,,,,,,,,,,,,,,,,,,,,,,Posture very good_x000D_\nVery tidy prep with ...,,Yes,Yes,No,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,Yes,Yes,No,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,No

In [70]:
# remove codes of the form 524_MCx and 578_MCx

def compare_supervisors(studentDf):
    if len(studentDf) != 2:
        print("Error: The student must have exactly two rows for comparison.")
        return None
    
    row1, row2 = studentDf.iloc[0], studentDf.iloc[1]
    
    # Identify columns where neither row has NA
    valid_cols = [col for col in assessormcCols if row1[col] != 'NA' and row2[col] != 'NA']
    
    # Filter to only Yes/No columns
    yes_no_cols = [col for col in valid_cols if studentDf[col].isin(['Yes', 'No']).all()]
    
    # Count differences
    diff_yes_to_no = sum(row1[col] == 'Yes' and row2[col] == 'No' for col in yes_no_cols)
    diff_no_to_yes = sum(row1[col] == 'No' and row2[col] == 'Yes' for col in yes_no_cols)
    
    # Count how many more Yes answers the first supervisor has over the second
    yes_first_more = sum(row1[col] == 'Yes' and row2[col] == 'No' for col in yes_no_cols) - \
                     sum(row1[col] == 'No' and row2[col] == 'Yes' for col in yes_no_cols)

    return {
        "Yes to No": diff_yes_to_no,
        "No to Yes": diff_no_to_yes,
        "Yes (First Supervisor) More By": yes_first_more
    }

def createAssessorSummary(df, assessormcCols):
    assessor_value_counts = {}
    for assessor in df['assessor_name'].unique():
        assessorDf = df[df['assessor_name'] == assessor]
        # Count occurrences of each unique value across all assessor MC columns
        value_counts = assessorDf[assessormcCols].apply(pd.Series.value_counts).sum(axis=1).to_dict()
        # add value for number of assessments
        value_counts['Total Forms'] = len(assessorDf)
        value_counts['entrustment'] = assessorDf['entrustment'].value_counts().to_dict()
        # Store result
        assessor_value_counts[assessor] = value_counts

    # Convert results to DataFrame for better visualization
    assessor_summary_df = pd.DataFrame.from_dict(assessor_value_counts, orient='index').fillna(0)
    assessor_summary_df['Total'] = assessor_summary_df.sum(axis=1)
    # total of Yes and No
    assessor_summary_df['YesNoTotal'] = assessor_summary_df['Yes'] + assessor_summary_df['No']
    assessor_summary_df['Leniency'] = assessor_summary_df['Yes'] / assessor_summary_df['YesNoTotal']
    # sort on linency
    assessor_summary_df.sort_values(by='Leniency', inplace=True, ascending=False)
    return assessor_summary_df


In [ ]:
code1 = '533'
code2 = '577'
def getAssessorAnalysis(df, codes, assessormcCols, studentmcCols):
    df = df.loc[:, ~df.columns.str.contains(f"{code1}_MC")]
    df = df.loc[:, ~df.columns.str.contains(f"{code2}_MC")]
    assessormcCols = [col for col in assessormcCols if code1 not in col and code2 not in col]
    studentmcCols = [col for col in studentmcCols if code1 not in col and code2 not in col]
    naCutoff = len(assessormcCols) // 2
    print(f'NA Cutoff: {naCutoff}')
    df['#YesStu'] = df[studentmcCols].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['#YesAss'] = df[assessormcCols].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['#NoStu'] = df[studentmcCols].apply(lambda x: (x == 'No').sum(), axis=1)
    df['#NoAss'] = df[assessormcCols].apply(lambda x: (x == 'No').sum(), axis=1)
    df.replace({'Not Assessed': 'NA', 'Not Reviewed': 'NA'}, inplace=True)
    df['#NAStu'] = df[studentmcCols].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['#NAAss'] = df[assessormcCols].apply(lambda x: (x == 'NA').sum(), axis=1)
    df["Assessor Yes Ratio"] = df["#YesAss"] / (df["#YesAss"] + df["#NoAss"])
    # Remove rows where the denominator is 0 (to avoid division by zero errors)
    df = df.dropna(subset=["Assessor Yes Ratio"])
    scoreCols = ['#YesStu', '#YesAss', '#NoStu', '#NoAss', '#NAStu', '#NAAss']
    students = df['student_name'].unique()
    df.to_excel(f'{folder}\\michaelData {"+".join(codes)}.xlsx', index=False)
    df.sort_values(by='assessor_name', inplace=True)

    # Loop through each unique assessor
    # Dictionary to store results
    assessors = df['assessor_name'].unique()

    # ignore if #NAAss is greater than 12
    dffull= df.copy()
    df = df[df['#NAAss'] <= naCutoff]
    assessor_summary_df = createAssessorSummary(df, assessormcCols)
    
    # create anon names for assessor
    # assessor_summary_df['AnonName'] = [f'Assessor {i+1}' for i in range(len(assessor_summary_df))]
    # anonNameDict = assessor_summary_df['AnonName'].to_dict()
    anonNameDict = {'Amy Thompson': 'Assessor 9',
    'Carl Lakkis': 'Assessor 2',
    'Grace Pang': 'Assessor 5',
    'Jo-Anne Cherry': 'Assessor 6',
    'Jyoti Anand': 'Assessor 4',
    'Madeline Townsend': 'Assessor 10',
    'Martin Tyas': 'Assessor 3',
    'Melinda Johansson': 'Assessor 8',
    'Michael Wylie': 'Assessor 7',
    'Michelle Luu': 'Assessor 1'}
    pprint(anonNameDict)

    assessor_summary_df['AnonName'] = assessor_summary_df.index.map(anonNameDict)
    df['AnonName'] = df['assessor_name'].map(anonNameDict)
    display(assessor_summary_df)
    assessor_summary_df.to_excel(f'{folder}\\assessor_summary{"+".join(codes)}.xlsx')
    
    # Plot bar chart of leniency
    plt.figure(figsize=(12, 6))
    sns.barplot(x=assessor_summary_df.index, y=assessor_summary_df['Leniency'])
    plt.xticks(range(len(assessor_summary_df)), assessor_summary_df.index.map(anonNameDict), rotation=45)
    for i, v in enumerate(assessor_summary_df['Leniency']):
        plt.text(i, v + 0.01, f'{v:.2f}', color='black', ha='center')
    plt.xticks(rotation=45)
    plt.xlabel("Assessor")
    plt.ylabel("Leniency")
    plt.title(f"Assessor Leniency {'+'.join(codes)}")
    plt.savefig(f'{folder}\\leniency bar {"+".join(codes)}.png')
    plt.show()

    # Plot histogram of Yes Ratio for each assessor
    plt.figure(figsize=(12, 6))
    bins = np.linspace(0, 1, 11)  # Define bins from 0 to 1
    num_assessors = len(assessors)
    color_palette = sns.color_palette("tab10", num_assessors)  # More varied and contrasting colors
    for i, assessor in enumerate(assessors):
        assessor_data = df[df["assessor_name"] == assessor]["Assessor Yes Ratio"]
        sns.kdeplot(assessor_data, linestyle="--", label=anonNameDict[assessor], color=color_palette[i])
    wholeData = df["Assessor Yes Ratio"]
    sns.kdeplot(wholeData, linestyle="-", label="Mean line", color='black')
    plt.xlim(0, 1)
    plt.xlabel("Yes Ratio (Yes / (Yes + No))")
    plt.ylabel("Frequency")
    plt.title(f"Distribution of Leniency ({'+'.join(codes)})")
    plt.legend()
    plt.savefig(f'{folder}\\leniency dist {"+".join(codes)}.png')
    plt.show()

    # Create subplots with adjusted height and consistent x-axis limits
    # create a list of assessors sorted by anon name digit
    sorted_assessors = assessor_summary_df['AnonName'].tolist()
    # sort on digit
    sorted_assessors = sorted(sorted_assessors, key=lambda x: int(re.search(r'\d+', x).group()))
    print(f'Sorted Assessors: {sorted_assessors}')
    num_assessors = len(sorted_assessors)
    fig, axes = plt.subplots(nrows=(num_assessors // 3) + 1, ncols=3, figsize=(15, 12))
    axes = axes.flatten()

    # Determine x-axis limits for consistency
    xlim_min, xlim_max = df["Assessor Yes Ratio"].min(), df["Assessor Yes Ratio"].max()

    for i, assessor in enumerate(sorted_assessors):
        ax = axes[i]
        assessor_data = df[df["AnonName"] == assessor]["Assessor Yes Ratio"]
        
        # Plot histogram
        sns.histplot(assessor_data, bins=10, kde=True, ax=ax, alpha=0.7)
        
        # Plot mean line
        mean_value = assessor_data.mean()
        ax.axvline(mean_value, color='black', linestyle='dashed', linewidth=1)
        
        # Set title with anonymized name
        ax.set_title(f"{assessor}")
        ax.set_xlabel("Yes Ratio")
        ax.set_ylabel("Frequency")

        # Set consistent x-axis limits
        ax.set_xlim(xlim_min, xlim_max)

    # Remove empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout
    plt.tight_layout()
    plt.suptitle(f"Assessor Leniency Subplots {'+'.join(codes)}")
    plt.savefig(f'{folder}\\leniency subplots {"+".join(codes)}.png')
    plt.show()

    # create a second summary df with dffull
    assessor_summary_df2 = createAssessorSummary(dffull, assessormcCols)
    assessor_summary_df2['AnonName'] = assessor_summary_df2.index.map(anonNameDict)
    # plot stacked bar chart of Yes, No, NA
    colorDict = {
        'Yes': '#4c72b0',             # Muted Blue
        'Yes with guidance': '#55a868',  # Soft Green
        'No': '#e07a5f',             # Warm Red-Orange
        'Not assessed': '#bfbfbf',    # Neutral Grey
        'NA': '#bfbfbf'              # Grey for renamed 'Not assessed'
    }
    colors = [colorDict[col] for col in ['Yes', 'No', 'NA']]
    fig, ax = plt.subplots(figsize=(12, 6)) 
    assessor_summary_df2[['Yes', 'No', 'NA']].plot(kind='bar', stacked=True, ax=ax, color=colors)
    plt.xticks(range(len(assessor_summary_df2)), assessor_summary_df2['AnonName'], rotation=45)
    plt.xlabel("Assessor")
    plt.ylabel("Count")
    plt.title(f"Assessor MC Counts {'+'.join(codes)}")
    plt.suptitle(f"Stacked bar chart {'+'.join(codes)}")
    plt.savefig(f'{folder}\\assessor MC counts {"+".join(codes)}.png')

getAssessorAnalysis(df.copy(), ['524', '578'], assessormcCols, studentmcCols)

NA Cutoff: 11


TypeError: unsupported operand type(s) for +: 'float' and 'dict'

## BOH2

In [ ]:
workbookpath = '2025\BOH2\\Form 1\\all_data.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
df.fillna('', inplace=True)
with open('2025\sim_checklists.json') as f:
    simChecklist = json.load(f)
    simChecklist = {item['key']: item for item in simChecklist['result']}     # can get full name by key fields MCx
with open('2025\clinic_checklists.json') as f:
    clinicChecklist = json.load(f)
    clinicChecklist = {item['key']: item for item in clinicChecklist['result']}

pprint(simChecklist, indent=4)
pprint(clinicChecklist, indent=4)
pecItems  = ['positioning', 'Consent', 'Record', 'infection']
rubricCols = ['time_mgmt', 'communication', 'professionalism', 'entrustment']
mcCols = [col for col in df.columns if 'MC' in col and 'positioning' not in col and 'Consent' not in col and 'Record' not in col and 'infection' not in col]
pecCols = [col for col in df.columns if 'positioning' in col or 'Consent' in col or 'Record' in col or 'infection' in col]
studentmcCols = [col for col in mcCols if 'student' in col]
assessormcCols = [col for col in mcCols if 'assessor' in col]

otherCols = [col for col in df.columns if col not in mcCols and col not in rubricCols and col not in pecCols]
print(otherCols)
otherfullnamedict = {'student_name': 'Student Name', 'assessor_name': 'Assessor Name', 'date': 'Date', 'cohort': 'Cohort', 'subject': 'Subject', 'type': 'Type',
                     'time_mgmt': 'Time Management', 'communication': 'Communication', 'professionalism': 'Professionalism', 'entrustment': 'Entrustment',
                     'student_feedback': 'Student Feedback', 'assessor_feedback': 'Assessor Feedback', 'clinical_incident': 'Clinical Incident',
                     'patient_complexity': 'Complexity', 'Consent': 'Consent'}
mcColDict = {}
for col in mcCols:
    key = col.split('_')[1]
    if key not in mcColDict:
        mcColDict[key] = []
    mcColDict[key].append(col)
def get_items(row):
    items = []
    for key, cols in mcColDict.items():
        if any((not pd.isna(row[col])) and (row[col] not in ['', 'nan']) for col in cols):
            if key in pecItems:
                continue
            items.append(key)
    return items

df['Items'] = df.apply(get_items, axis=1)
display(df.head())

In [65]:
def createStudentReport(df, studentName):
    elements = []
    studentdf = df[df['student_name'] == studentName]
    # sort by date
    studentdf['date'] = pd.to_datetime(studentdf['date'])
    studentdf.sort_values('date', inplace=True)
    studentdf['date'] = studentdf['date'].dt.strftime('%d-%m-%Y')
    os.makedirs(f'{folder}\\Student Reports', exist_ok=True)
    doc = SimpleDocTemplate(f'{folder}\\Student Reports\\{studentName}.pdf', pagesize= pageSize, leftMargin = leftMargin,
                            rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    
    # first heading
    elements.append(Paragraph(f'{studentName}', headingStyle))
    elements.append(Spacer(1, 24))

    # get the forms
    for i, (idx, row) in enumerate(studentdf.iterrows()):
        formitems = row['Items']
        type = row['type']
        elements.append(Paragraph(f'Form {i+1}: {row[colDate.lower()]}', subheadingStyle))
        elements.append(Spacer(1, 12))
        # elements.append(Paragraph(f'Role: {row[colRole]}', subsubheadingStyle))
        if len(formitems) > 0:
            elements.append(Spacer(1, 12))
            elements.append(Paragraph('Items Performed', subsubheadingStyleL))
            elements.append(Spacer(1, 12))
            elements.append(Paragraph(', '.join(formitems), tableTextStyleL))
        elements.append(Spacer(1, 12))
        # add other columns
        for col in otherCols:
            if pd.isna(row[col]) or row[col] == 'nan' or row[col] == '':
                continue
            # print(f'{col}: {row[col]}')
            elements.append(Paragraph(f'{otherfullnamedict[col]}', smallsubsubheadingStyleL))
            elements.append(Spacer(1, 6))
            # text = row[col]
            text = row[col].strip().replace('\n', '<br/>')
            # remove _x000D_
            text = re.sub(r'_x000D_', '', text)
            # remove non-ascii characters
            text = re.sub(r'[^\x00-\x7F]+', '', text)
            elements.append(Paragraph(f'{text}', tableTextStyleL))
            elements.append(Spacer(1, 12))


        rubricDf = pd.DataFrame(columns = ['Rubric', 'Score'])
        for col in rubricCols:
            rubricDf = pd.concat([rubricDf, pd.DataFrame({'Rubric': [otherfullnamedict[col]], 'Score': [row[col]]})])
        rubricTable = createTable(rubricDf, 'Rubric Scores', colRatio=[1, 3], customTextCols=[0, 1], tableTextStyle=tableTextStyleSmall)
        # alter top and bottom padding
        table = rubricTable._content[2]
        table.setStyle(TableStyle([
            ('TOPPADDING', (0, 0), (-1, -1), 7),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 7),
        ]))
        elements.append(rubricTable)
        elements.append(Spacer(1, 12))
        # add mc col table
        for item in formitems:
            print(item, type)
            stumcCol = [col for col in studentmcCols if item in col]
            assmcCol = [col for col in assessormcCols if item in col]
            if type == 'Simulation':
                checklist = simChecklist[item]
                itemName = simChecklist[item]['name']
            else:
                checklist = clinicChecklist[item]
                itemName = clinicChecklist[item]['name']
            if len(studentmcCols) == 0 or len(assessormcCols) == 0:
                continue
            evaldf = pd.DataFrame(columns = ['Marking Checklist', 'Student', 'Educator'])
            for stucol, educol in zip(stumcCol, assmcCol):
                qno = stucol.split('_')[-1]
                markingchecklist = checklist['fields'][qno]
                student = row[stucol]
                educator = row[educol]
                evaldf = pd.concat([evaldf, pd.DataFrame({'Marking Checklist': [markingchecklist], 'Student': [student], 'Educator': [educator]})])
            
            evaldf.replace({'Not Assessed': 'NA', 'Not Reviewed': 'NA', 'nan': 'NA'}, inplace=True)
            evalTable = createTable(evaldf, f'Evaluation for {item}: {itemName}', colRatio = [5, 1, 1], customTextCols=[0], cellHighlight=True, tableTextStyle=tableTextStyleSmall)
            # alter top and bottom padding
            table = evalTable._content[2]
            # if it's a table
            if isinstance(table, Table):
                table.setStyle(TableStyle([
                    ('TOPPADDING', (0, 0), (-1, -1), 7),
                    ('BOTTOMPADDING', (0, 0), (-1, -1), 7),
                ]))
            elements.append(evalTable)
            elements.append(Spacer(1, 12))
        # add pec table
        # pecdf = pd.DataFrame(columns = ['Marking Checklist', 'Student', 'Educator'])
        # for col in pecCols:
        #     pecdf = pd.concat([pecdf, pd.DataFrame({'Marking Checklist': [fullnamedict[col].split(' - ')[-1]], 'Student': [row[col]], 'Educator': [row[col.replace('#1', '#2')]]})])
        # pecdf.replace({'Not Assessed': 'NA', 'Not Reviewed': 'NA', 'nan': 'NA'}, inplace=True)
        # pecTable = createTable(pecdf, 'Other Marking Checklist', colRatio = [5, 1, 1], customTextCols=[0], cellHighlight=True, tableTextStyle=tableTextStyleSmall)
        # elements.append(pecTable)
        # table = pecTable._content[2]
        # table.setStyle(TableStyle([
        #     ('TOPPADDING', (0, 0), (-1, -1), 7),
        #     ('BOTTOMPADDING', (0, 0), (-1, -1), 7),
        # ]))
        elements.append(PageBreak())
    doc.build(elements)

for studentname in df['student_name'].unique():
    print(studentname)
    createStudentReport(df, studentname)
    # break

Bella Dong
Creating table for Rubric Scores
533 Simulation
Creating table for Evaluation for 533: Adhesive restoration - three surfaces - posterior tooth - direct
577 Simulation
Creating table for Evaluation for 577: Cusp Capping per Cusp
Creating table for Rubric Scores
533 Simulation
Creating table for Evaluation for 533: Adhesive restoration - three surfaces - posterior tooth - direct
577 Simulation
Creating table for Evaluation for 577: Cusp Capping per Cusp
Creating table for Rubric Scores
524 Simulation
Creating table for Evaluation for 524: Adhesive restoration - four surfaces - anterior tooth - direct
578 Simulation
Creating table for Evaluation for 578: Restoration of an incisal corner
Creating table for Rubric Scores
524 Simulation
Creating table for Evaluation for 524: Adhesive restoration - four surfaces - anterior tooth - direct
578 Simulation
Creating table for Evaluation for 578: Restoration of an incisal corner
Mehrana Rahimpour
Creating table for Rubric Scores
533 Simu

In [66]:
plt.ioff()
colorDict = {
    'Yes': '#4c72b0',             # Muted Blue
    'Yes with guidance': '#55a868',  # Soft Green
    'No': '#e07a5f',             # Warm Red-Orange
    'Not assessed': '#bfbfbf',    # Neutral Grey
    'NA': '#bfbfbf'              # Grey for renamed 'Not assessed'
}

def getYesNo(df, stuCols, supCols):
    df['YesStu'] = df[stuCols].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['NoStu'] = df[stuCols].apply(lambda x: (x == 'No').sum(), axis=1)
    df['NAStu'] = df[stuCols].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['ScoreStu'] = df['YesStu']/(df['YesStu'] + df['NoStu'] + 0.00001)
    df['YesSup'] = df[supCols].apply(lambda x: (x == 'Yes').sum(), axis=1)
    df['NoSup'] = df[supCols].apply(lambda x: (x == 'No').sum(), axis=1)
    df['NASup'] = df[supCols].apply(lambda x: (x == 'NA').sum(), axis=1)
    df['ScoreSup'] = df['YesSup']/(df['YesSup'] + df['NoSup'] + 0.00001)
    df['ScoreDiff'] = df['ScoreStu'] - df['ScoreSup']
    return df
colId = 'student_name'

def plotStackedYesNo(df, colId, title):
    plotdf = df[[colId, 'YesStu', 'NoStu', 'NAStu', 'YesSup', 'NoSup', 'NASup']]
    # sort by highest NoSup
    # display(plotdf)
    plotdf = plotdf.groupby(colId).sum()
    plotdf.sort_values(by='NoSup', ascending=True, inplace=True)
    # display(plotdf)
    # plotdf.set_index(colId, inplace=True)
    studf = plotdf[['YesStu', 'NoStu', 'NAStu']]
    studf.columns = ['Yes', 'No', 'NA']
    supdf = plotdf[['YesSup', 'NoSup', 'NASup']]
    supdf.columns = ['Yes', 'No', 'NA']
    colors = [colorDict[key] for key in ['Yes', 'No', 'NA']]
    fig, ax = plt.subplots(1, 2, figsize=(figSize[0], figSize[1]))
    studf.plot(kind='barh', stacked=True, ax=ax[0], color=colors)
    ax[0].set_title('Student')
    ax[0].set_ylabel('')
    supdf.plot(kind='barh', stacked=True, ax=ax[1], color=colors)
    ax[1].set_title('Supervisor')
    ax[1].set_ylabel('')    
    # hide legend for first plot
    ax[0].get_legend().remove()
    ax[1].legend(title='Evaluation', bbox_to_anchor=(1.05, 1.02))
    # add horizontal space
    plt.tight_layout()
    fig.suptitle(title, y = 1.02)
    img = addPlotImage(fig, 0.9)
    return img

uniquevals = [ 'Yes','No', 'NA']
# colors = {'Yes': 'blue', 'No': 'orange', 'Not assessed': 'lightgrey', 'Not relevant': 'green'}
colors = ['blue', 'orange', 'lightgrey']
def figToImage(fig):
    """Converts a Matplotlib figure to a PIL Image"""
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    return PIL.Image.open(buf)

def getEvalCounts(df, colGeneralEval, checklist, item):
    # create df for evaluation counts for each column
    evalcountDf = pd.DataFrame(columns = ['Question'] + uniquevals)
    for col in colGeneralEval:
        evalcount = df[col].value_counts()
        # remove ''
        evalcount = evalcount.drop('', errors='ignore')
        if len(evalcount) ==0:
            return None
        print(evalcount)
        qno = col.split('_')[-1]
        if qno in checklist[item]['fields']:
            fulltext = checklist[item]['fields'][qno]
        else:
            fulltext = 'Not Found'
        evalcountDf = pd.concat([evalcountDf, pd.DataFrame({'Question': [qno],
                                                             **evalcount.to_dict()})])
    evalcountDf.fillna(0, inplace=True)
    # sort by the total sum of the columns
    evalcountDf['sum'] = evalcountDf[uniquevals].sum(axis=1)
    evalcountDf = evalcountDf.sort_values('sum', ascending=False)
    # remove rows with sum = 0
    evalcountDf = evalcountDf[evalcountDf['sum'] != 0]
    evalcountDf.drop('sum', axis=1, inplace=True)
    # sort by question where question is MC\d+ use the digit
    evalcountDf['Question Num'] = evalcountDf['Question'].str.extract(r'(\d+)', expand=False).astype(int)
    evalcountDf = evalcountDf.sort_values('Question Num')
    evalcountDf.drop('Question Num', axis=1, inplace=True)
    # if max value is less than 2 then return None
    if evalcountDf[uniquevals].max().max() < 2:
        return None
    evalcountDf = evalcountDf.head(15)
    return evalcountDf

# create stacked bar chart for general evaluation counts
def createStackedBarChart(df, title, plotCol):
    fig, ax = plt.subplots(figsize=(figSize[0]*0.6, figSize[1]*0.8))
    plotdf = df.set_index(plotCol)
    plotdf = plotdf.loc[::-1]  # Reverse the order of bars
    if len(plotdf) == 0:
        return None
    # remove rows with all zeros
    plotdf = plotdf.loc[(plotdf!=0).any(axis=1)]
    
    plotdf.plot(kind='barh', stacked=True, ax=ax, color=colors)
    ax.set_title(title)
    ax.set_xlabel('Questions')
    ax.set_ylabel('Counts')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=8)
    ax.legend(loc='center left', bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()
    return fig


In [67]:
# overall report
df.replace({'nan': 'NA', 'Not Assessed': "NA", "Not Reviewed": "NA"}, inplace=True)
elements = []
doc = SimpleDocTemplate(f'{folder}\\Overall Report.pdf', pagesize= pageSize, leftMargin = leftMargin,
                        rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
elements.append(Paragraph(f'Overall Report (28-02-2025)', headingStyle))
elements.append(Spacer(1, 24))
nforms = len(df)
typecounts = df['type'].value_counts()
entrustmentcounts = df['entrustment'].value_counts().drop('', errors='ignore')
entrustmentcounts = entrustmentcounts.sort_index()
# item counts
allitems = []
for idx, row in df.iterrows():
    allitems.extend(row['Items'])
counts = pd.Series(allitems).value_counts()
itemcounts = counts.to_dict()

# create a table for the counts
countdf = pd.DataFrame(columns = ['Metric', 'Value'])
countdf = pd.concat([countdf, pd.DataFrame({'Metric': ['Number of Forms'], 'Value': [nforms]})])
typecountText = '<br/> '.join([f'{key}: {value}' for key, value in typecounts.items()])
countdf = pd.concat([countdf, pd.DataFrame({'Metric': ['Type Counts'], 'Value': [typecountText]})])
entrustmentText = '<br/> '.join([f'{int(key)}: {value}' for key, value in entrustmentcounts.items()])
itemcountText = '<br/> '.join([f'{key}: {value}' for key, value in itemcounts.items()])
countdf = pd.concat([countdf, pd.DataFrame({'Metric': ['Entrustment Counts'], 'Value': [entrustmentText]})])
countdf = pd.concat([countdf, pd.DataFrame({'Metric': ['Item Counts'], 'Value': [itemcountText]})])
countTable = createTable(countdf, 'Summary Table', colRatio=[3, 1], customTextCols=[0, 1], tableTextStyle=tableTextStyleLarge)
elements.append(countTable)
elements.append(PageBreak())
# create a student name and items table
studentItemDf = df[[colId, 'Items']]
#aggregate items with combining the item lists
studentItemDf = studentItemDf.groupby(colId).agg({'Items': 'sum'}).reset_index()

studentItemDf['Items'] = studentItemDf['Items'].apply(lambda x: list(set(x)))
studentItemDf['Items'] = studentItemDf['Items'].apply(lambda x: ', '.join(x))
studentItemDf = studentItemDf.sort_values(colId)
studentItemTable = createTable(studentItemDf, 'Student Items Table', colRatio=[2, 2], customTextCols=[0, 1], tableTextStyle=tableTextStyleLarge)
# elements.append(studentItemTable)
# elements.append(PageBreak())

# get all the items

for type_ in df['type'].unique():
    print(type_)
    thisdf = df[df['type'] == type_]
    allitems = []
    for idx, row in thisdf.iterrows():
        allitems.extend(row['Items'])
    allitems = list(set(allitems))
    elements.append(Paragraph(f'{type_}', subheadingStyle))
    elements.append(Spacer(1, 12))
    checklist = simChecklist if type_ == 'Simulation' else clinicChecklist
    # create stackec bar chart of value counts for each column for ech item
    for item in allitems:
        print(item, checklist[item]['name'])

        stuCols = [col for col in studentmcCols if item in col]
        supCols = [col for col in assessormcCols if item in col]
        # itemdf = getYesNo(thisdf, stuCols, supCols)
        # img = plotStackedYesNo(itemdf, colId, item)
        # elements.append(img)
        evalcountDfStu = getEvalCounts(thisdf, stuCols, checklist, item)
        evalcountDfSup = getEvalCounts(thisdf, supCols, checklist, item)
        if evalcountDfStu is None or evalcountDfSup is None:
            continue
        # display(evalcountDf.head())
        figStu = createStackedBarChart(evalcountDfStu, 'Student', 'Question')
        figSup = createStackedBarChart(evalcountDfSup, 'Supervisor', 'Question')
        if figStu is None or figSup is None:
            continue
        imgSup = figToImage(figSup)
        imgStu = figToImage(figStu)
        fig, axes = plt.subplots(1, 2, figsize=(figSize[0], figSize[1]))
        axes[0].imshow(imgStu)
        axes[0].axis('off')
        axes[1].imshow(imgSup)
        axes[1].axis('off')
        plt.tight_layout()
        
        img = addPlotImage(fig, 0.9)
        elements.append(Paragraph(f'{item}: {checklist[item]["name"]}', subheadingStyle))
        elements.append(Spacer(1, 12))
        elements.append(img)
        # create a full text reference table
        fulltextdf = pd.DataFrame(columns = ['Question', 'Full Text'])
        for qno, text in checklist[item]['fields'].items():
            fulltextdf = pd.concat([fulltextdf, pd.DataFrame({'Question': [qno], 'Full Text': [text]})])
        fulltextdf['Question Num'] = fulltextdf['Question'].str.extract(r'(\d+)', expand=False).astype(int)
        fulltextdf = fulltextdf.sort_values('Question Num')
        fulltextdf.drop('Question Num', axis=1, inplace=True)
        toppadding = 6
        bottompadding = 6
        if len(fulltextdf) > 10:
            toppadding = 3
            bottompadding = 3
        fulltextTable = createTable(fulltextdf, 'MC Reference Table', colRatio=[1, 3], customTextCols=[0, 1], tableTextStyle=tableTextStyleSmall,
                                    tableWidth=0.9, topPadding=toppadding, bottomPadding=bottompadding)
        elements.append(fulltextTable)
        elements.append(PageBreak())

doc.build(elements)

Creating table for Summary Table
Creating table for Student Items Table
Simulation
533 Adhesive restoration - three surfaces - posterior tooth - direct
student_533_MC1
Yes    161
No      19
NA       4
Name: count, dtype: int64
student_533_MC2
Yes    93
NA     61
No     30
Name: count, dtype: int64
student_533_MC3
Yes    87
NA     63
No     34
Name: count, dtype: int64
student_533_MC4
Yes    103
NA      62
No      19
Name: count, dtype: int64
student_533_MC5
Yes    76
NA     57
No     51
Name: count, dtype: int64
student_533_MC6
NA     95
Yes    65
No     24
Name: count, dtype: int64
student_533_MC7
NA     98
Yes    83
No      3
Name: count, dtype: int64
student_533_MC8
NA     110
No      53
Yes     21
Name: count, dtype: int64
student_533_MC9
NA     116
No      42
Yes     26
Name: count, dtype: int64
student_533_MC10
NA     115
No      35
Yes     34
Name: count, dtype: int64
student_533_MC11
NA     114
Yes     52
No      17
Name: count, dtype: int64
student_533_MC12
NA     119
Yes     

### Standard Setting for Aditi

In [68]:
##
# Reload necessary libraries after execution state reset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Reload the Preparation and Restoration scores
file_path = "2025\Aditi\Cleaned_Transposed_Workshopping_Checklist.xlsx"
folder, file, ext = getFolderandFileName(file_path)
xls = pd.ExcelFile(file_path)

# Load data
prep_df = xls.parse("Preparation")
rest_df = xls.parse("Restoration")

# Reload the importance ranking and difficulty tables
prep_data = {
    "Item": [
        "DEJ is caries-free *",
        "Principles of selective caries removal on pulpal surfaces satisfied *",
        "Sufficient and uniform layer of carious dentine remaining, limited to pulpal surfaces only",
        "Conservative in the mesio-distal dimension",
        "Conservative in the bucco-lingual dimension",
        "Depth not overextended nor less than 1.5mm",
        "Avoids excessive loss or weakening of tooth structure or compromising pulp space *",
        "Gingival contact cleared *",
        "Cavosurface margins free from irregularities and small areas of undermined enamel",
        "Free from large areas of unsupported enamel or large fractures*",
        "Internal surfaces smooth and rounded without minor roughness",
        "Internal surfaces smooth and rounded without obvious irregularities and/or gouges",
        "No or minimal scuffs/ scratches/ surface loss",
        "Avoids small damages to adjacent tooth and/or surrounding surfaces requiring tooth modification",
        "Avoids large damages to adjacent tooth and/or surrounding surfaces requiring restoration *"
    ],
    "Difficulty": [
        "HARD", "HARD", "HARD", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE",
        "HARD", "HARD", "HARD", "HARD", "MODERATE", "MODERATE", "MODERATE"
    ],
    "Importance Ranking": [10, 10, 6, 7, 7, 7, 10, 10, 4, 10, 2, 5, 2, 7, 10]
}

rest_data = {
    "Item": [
        "Smooth and well-polished",
        "Sub-surface porosities or voids absent",
        "Large porosities or voids absent*",
        "Smooth junction",
        "Free of minor excess/deficiency only detectable by dental explorer",
        "Free of major excess that warrants additional finishing and polishing",
        "No open margins *",
        "No overhangs that warrant significant finishing and polishing*",
        "Tooth anatomy recreated",
        "Contour not bulky (including marginal ridge)",
        "Contour not deficient (including marginal ridge)",
        "Establishes anatomical contact area *",
        "No or minimal scuffs/ scratches/ surface loss",
        "Avoids surface damage 2mm beyond restoration",
        "Avoids damage to adjacent tooth and/or surrounding surfaces requiring restorative correction *"
    ],
    "Difficulty": [
        "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE",
        "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "HARD", "HARD", "HARD"
    ],
    "Importance Ranking": [2, 7, 10, 8, 8, 8, 10, 10, 2, 5, 7, 10, 2, 5, 10]
}

weight_prep = {"critical": 16, "major": 2, "minor": 1}
cutoff_prep = 0.86
weight_resto = {"critical": 25, "major": 3, "minor": 1}
cutoff_resto = 0.85
# critical are with importance 10, major with 5-7, else minor

prep_criteria_df = pd.DataFrame(prep_data)
rest_criteria_df = pd.DataFrame(rest_data)

# given the importance ranking, assign the weight
def assign_weight(importance, weight):
    if importance == 10:
        return weight["critical"]
    elif importance >= 5:
        return weight["major"]
    else:
        return weight["minor"] 

prep_criteria_df["Weight"] = prep_criteria_df["Importance Ranking"].apply(lambda x: assign_weight(x, weight_prep))
rest_criteria_df["Weight"] = rest_criteria_df["Importance Ranking"].apply(lambda x: assign_weight(x, weight_resto))
display(prep_criteria_df)
display(rest_criteria_df)






,Item,Difficulty,Importance Ranking,Weight
0,DEJ is caries-free *,HARD,10,16
1,Principles of selective caries removal on pulp...,HARD,10,16
2,Sufficient and uniform layer of carious dentin...,HARD,6,2
3,Conservative in the mesio-distal dimension,MODERATE,7,2
4,Conservative in the bucco-lingual dimension,MODERATE,7,2
5,Depth not overextended nor less than 1.5mm,MODERATE,7,2
6,Avoids excessive loss or weakening of tooth st...,MODERATE,10,16
7,Gingival contact cleared *,MODERATE,10,16
8,Cavosurface margins free from irregularities a...,HARD,4,1
9,Free from large areas of unsupported enamel or...,HARD,10,16


,Item,Difficulty,Importance Ranking,Weight
0,Smooth and well-polished,MODERATE,2,1
1,Sub-surface porosities or voids absent,MODERATE,7,3
2,Large porosities or voids absent*,MODERATE,10,25
3,Smooth junction,MODERATE,8,3
4,Free of minor excess/deficiency only detectabl...,MODERATE,8,3
5,Free of major excess that warrants additional ...,MODERATE,8,3
6,No open margins *,MODERATE,10,25
7,No overhangs that warrant significant finishin...,MODERATE,10,25
8,Tooth anatomy recreated,MODERATE,2,1
9,Contour not bulky (including marginal ridge),MODERATE,5,3


In [69]:
# Calculate the total score for each student based on the weight of each criteria that are columns in the DataFrame
def calculate_total_score(df, criteria_df, cutoff, subject = "Preparation"):
    for i, row in df.iterrows():
        df.loc[i, "Total Score"] = sum([row[criteria] * weight for criteria, weight in zip(criteria_df["Item"], criteria_df["Weight"])])
    # divide  by the sum of the weights
    weightSum = criteria_df["Weight"].sum()
    print(weightSum)
    df["Total Score"] = df["Total Score"] / criteria_df["Weight"].sum()
    # determine if the student passed or failed
    df["Pass/Fail"] = df["Total Score"].apply(lambda x: "Pass" if x >= cutoff else "Fail")
    # student column has Student ids as Student 1, Student 2, etc. sort by the number and then by examiner
    df["StudentID"] = df["Student"].apply(lambda x: int(x.split()[1]))
    df["ExaminerID"] = df["Examiner"].apply(lambda x: int(x.split()[1]))
    df.sort_values(by=["StudentID", "ExaminerID"], inplace=True)

    df.drop("ExaminerID", inplace=True, axis=1)

    # Create bar plot of the total score and the pass/fail status for each student, plotted side by side for each examiner
    fig, ax = plt.subplots(1, 1, figsize=(24, 6))
    sns.barplot(x="Student", y="Total Score", hue="Examiner", data=df, dodge=True, ax=ax, width=0.8)
    ax.set_title(f"Total Score for Each Student {subject}")
    ax.set_xlabel("Student")
    ax.set_ylabel("Total Score")
    ax.set_ylim(0, 1.2)
    # Show score as text above each bar
    for p in ax.patches:
        height = p.get_height()
        if height > 0:  # Only annotate bars with positive height
            ax.annotate(f"{height:.2f}", 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha="center", va="center", 
                        xytext=(0, 5), textcoords="offset points", fontsize=8)
        
    # Add a horizontal line at the cutoff
    ax.axhline(y=cutoff, color="red", linestyle="--", label='Cutoff')
    plt.legend(loc='upper right')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{folder}\{subject} Total Score.png")

    # Create a bar plot of the # passed and # failed students for each examiner
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    sns.countplot(x="Examiner", hue="Pass/Fail", data=df, dodge=True, ax=ax)
    ax.set_title(f"Number of Students Passed/Failed {subject}")
    ax.set_xlabel("Examiner")
    ax.set_ylabel("Count")

    # Show count as text above each bar
    for p in ax.patches:
        height = p.get_height()
        if height > 0:  # Only annotate bars with positive height
            ax.annotate(f"{int(height)}", 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha="center", va="center", 
                        xytext=(0, 5), textcoords="offset points", fontsize=8)

    plt.legend(title='Pass/Fail', loc='upper right')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{folder}\{subject} Pass Fail.png")


    df.drop("StudentID", inplace=True, axis=1)

    return df

prep_df = calculate_total_score(prep_df, prep_criteria_df, cutoff_prep, subject="Preparation")
rest_df = calculate_total_score(rest_df, rest_criteria_df, cutoff_resto, subject="Restoration")
display(prep_df.head())
display(rest_df.head())

# save the data to an Excel file with two sheets
output_file = f"{folder}\{file}_Total_Score.xlsx"
# with pd.ExcelWriter(output_file) as writer:
    # prep_df.to_excel(writer, sheet_name="Preparation", index=False)
    # rest_df.to_excel(writer, sheet_name="Restoration", index=False)

111
149


,Student,Examiner,DEJ is caries-free *,Principles of selective caries removal on pulpal surfaces satisfied *,"Sufficient and uniform layer of carious dentine remaining, limited to pulpal surfaces only",Conservative in the mesio-distal dimension,Conservative in the bucco-lingual dimension,Depth not overextended nor less than 1.5mm,Avoids excessive loss or weakening of tooth structure or compromising pulp space *,Gingival contact cleared *,Cavosurface margins free from irregularities and small areas of undermined enamel,Free from large areas of unsupported enamel or large fractures*,Internal surfaces smooth and rounded without minor roughness,Internal surfaces smooth and rounded without obvious irregularities and/or gouges,No or minimal scuffs/ scratches/ surface loss,Avoids small damages to adjacent tooth and/or surrounding surfaces requiring tooth modification,Avoids large damages to adjacent tooth and/or surrounding surfaces requiring restoration *,Total,Total Score,Pass/Fail
0,Student 1,Examiner 1,1,1,0,1,0,1,1,1,1,1,1,1,0,0,1,11,0.936937,Pass
20,Student 1,Examiner 2,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,11,0.945946,Pass
40,Student 1,Examiner 3,1,1,0,1,1,1,0,1,1,1,0,1,0,0,1,10,0.801802,Fail
11,Student 2,Examiner 1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,1,6,0.351351,Fail
31,Student 2,Examiner 2,0,0,0,1,1,1,0,1,0,0,0,0,1,1,1,7,0.369369,Fail


,Student,Examiner,Smooth and well-polished,Sub-surface porosities or voids absent,Large porosities or voids absent*,Smooth junction,Free of minor excess/deficiency only detectable by dental explorer,Free of major excess that warrants additional finishing and polishing,No open margins *,No overhangs that warrant significant finishing and polishing*,Tooth anatomy recreated,Contour not bulky (including marginal ridge),Contour not deficient (including marginal ridge),Establishes anatomical contact area *,No or minimal scuffs/ scratches/ surface loss,Avoids surface damage 2mm beyond restoration,Avoids damage to adjacent tooth and/or surrounding surfaces requiring restorative correction *,Total,Total Score,Pass/Fail
0,Student 1,Examiner 1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,12,0.939597,Pass
20,Student 1,Examiner 2,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,12,0.939597,Pass
40,Student 1,Examiner 3,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,10,0.751678,Fail
11,Student 2,Examiner 1,0,0,1,0,0,0,1,1,0,0,1,1,1,1,1,8,0.885906,Pass
31,Student 2,Examiner 2,0,0,1,0,0,0,1,1,0,1,1,1,1,1,1,9,0.906040,Pass
